# Kubernetes Management Tasks

This is a demo notebook for common kubernetes tasks.

*Note: You should be aware of where and how the jupyter session is running, as kubernetes credentials are sensitive information and leakage posts security risks.*

## Configure kubectl

If you did not start this notebook with pre-configured KUBECONFIG, you may paste your `kubectl config view --raw --minify` output here.  Otherwise skip to the next section.

Notes:
1. if you use gke, your `kubectl config` contains a token that a valid for an hour. Run `kubectl get cs` to refresh token locally before copying the config.

In [26]:
import ipywidgets
from kubeconfig import KubeConfig
k = KubeConfig()
k.test()

In [27]:
k.setup()

Kubernetes master is running at https://10.63.240.1:443


Valid(value=True, description='Use existing service account.', layout=Layout(width='100%'), style=DescriptionS…

## Check version and cluster health

In [28]:
!! kubectl version

['Client Version: version.Info{Major:"1", Minor:"12", GitVersion:"v1.12.7", GitCommit:"6f482974b76db3f1e0f5d24605a9d1d38fad9a2b", GitTreeState:"clean", BuildDate:"2019-03-25T02:52:13Z", GoVersion:"go1.10.8", Compiler:"gc", Platform:"linux/amd64"}',
 'Server Version: version.Info{Major:"1", Minor:"12+", GitVersion:"v1.12.7-gke.25", GitCommit:"d4c79083ab6dea5d26ef4ed8d50b145268349bc3", GitTreeState:"clean", BuildDate:"2019-06-22T16:10:31Z", GoVersion:"go1.10.8b4", Compiler:"gc", Platform:"linux/amd64"}']

In [29]:
!! kubectl get cs

['NAME                 STATUS    MESSAGE              ERROR',
 'scheduler            Healthy   ok                   ',
 'controller-manager   Healthy   ok                   ',
 'etcd-0               Healthy   {"health": "true"}   ',
 'etcd-1               Healthy   {"health": "true"}   ']

In [30]:
!! kubectl get node

['NAME                                          STATUS   ROLES    AGE    VERSION',
 'gke-cluster-kent-default-pool-005ef893-8rfq   Ready    <none>   109d   v1.12.5-gke.5',
 'gke-cluster-kent-default-pool-005ef893-fg7d   Ready    <none>   2d1h   v1.12.5-gke.5',
 'gke-cluster-kent-default-pool-005ef893-sv1w   Ready    <none>   109d   v1.12.5-gke.5']

## Delete user pvc
Run the cell to render a dropdown list, select user pvc for deletion.

In [31]:
user_pvcs = !! kubectl get -l component=singleuser-storage  pvc -n hub -o jsonpath='{range.items[*]}{.metadata.name}{"\n"}{end}'

pvc = ipywidgets.Dropdown(
    options=user_pvcs,
    description='Target:',
    disabled=False,
)
pvc

Dropdown(description='Target:', options=('claim-phadmin', 'claim-phuser-2d1', 'claim-phuser-2d2', 'claim-phuse…

Run the cell to execute the deletion.

In [32]:
def execute():
    if pvc.value is None:
        print("No selected PVC")
        return

    mounted_by = !! ~/bin/kubectl -n hub describe pvc {pvc.value} | grep -o 'jupyter-.*'

    if mounted_by:
        print("PVC %s can't be deleted because it's mounted by: %s" % (pvc.value, mounted_by[0]))
    else:
        result = !! ~/bin/kubectl -n hub delete pvc {pvc.value}
        print(result)

execute()

['persistentvolumeclaim "claim-phadmin" deleted']


## Create pv-type dataset
Run the cell to render a dropdown list, select Target where to create a dataset.

In [33]:
pv_type_datasets = !! kubectl -n hub get datasets -o=custom-columns=VOLUME_NAME:spec.volumeName,NAME:metadata.name,TYPE:spec.type | grep -e 'pv$' | grep -v -e '^hostpath:' | awk '{print $1","$2}'
temp_datasets = {}
for ds in pv_type_datasets:
    [volume_name, name] = ds.split(',')
    temp_datasets[name] = volume_name
dataset_name = ipywidgets.Dropdown(
    options=temp_datasets.keys(),
    description='Target:',
    disabled=False,
)
dataset_name

Dropdown(description='Target:', options=('foo',), value='foo')

Run the cell to render a input field of `dataset size`, specify a size for the dataset which we want to create.

In [34]:
dataset_size = ipywidgets.IntText(description='Size:', value=200)
dataset_size

IntText(value=200, description='Size:')

Run the cell to determine the `storage_class`.

In [35]:
is_rook_block = !! kubectl get sc | grep rook-block

storage_class = 'rook-block' if is_rook_block else 'standard'
storage_class

'standard'

Run the cell to generate a yaml string of configuration.

In [36]:
dataset_volume = temp_datasets[dataset_name.value]

pv_type_dataset_yaml_string = '''apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  annotations:
    primehub-group: dataset-%s
    primehub-group-sc: %s
  name: dataset-%s
  namespace: hub
spec:
  accessModes:
  - ReadWriteMany
  dataSource: null
  resources:
    requests:
      storage: %s
  selector:
    matchLabels:
      primehub-group: dataset-%s
      primehub-namespace: hub
  storageClassName: ''
''' % (dataset_volume, storage_class, dataset_volume, str(dataset_size.value)+'Gi', dataset_volume)
print(pv_type_dataset_yaml_string)

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  annotations:
    primehub-group: dataset-foo-2
    primehub-group-sc: standard
  name: dataset-foo-2
  namespace: hub
spec:
  accessModes:
  - ReadWriteMany
  dataSource: null
  resources:
    requests:
      storage: 200Gi
  selector:
    matchLabels:
      primehub-group: dataset-foo-2
      primehub-namespace: hub
  storageClassName: ''



Run the cell to apply generated yaml for dataset creation.

In [37]:
!! echo "{pv_type_dataset_yaml_string}" | kubectl -n hub apply -f -

['persistentvolumeclaim/dataset-foo-2 unchanged']

## Resize Group Volume
Run the cell to render a dropdown list, select group volume to resize

In [1]:
import ipywidgets
from usage import *
from resize import *

group_volume = ipywidgets.Dropdown(
    options=get_group_volume_list().keys(),
    description='Group vol:',
    disabled=False,
)

group_volume

Dropdown(description='Group vol:', options=('data-nfs-dataset-foo-2-0', 'data-nfs-dummy-1-0', 'data-nfs-dummy-…

Run the cell to render a input field of group volume size, specify a size for the group volume which we want to resize.

In [9]:
usage = get_group_volume_usages(group_volume.value)

num, unit = usage.get(group_volume.value).get('usage').get('data')

new_usage = ipywidgets.BoundedIntText(    
    value=num,
    min=num,
    max=9999,
    step=1,
    description='New Size:',
    disabled=False)

def resize_handler(self):
    resize_group_volume(group_volume.value, str(new_usage.value) + unit)

resize_btn = ipywidgets.Button(description="Update", button_style="danger")
resize_btn.on_click(resize_handler)

ipywidgets.HBox([
    ipywidgets.Label(value="Current Size: %s %s" % (num, unit)),
    new_usage, 
    ipywidgets.Label(value=unit),
    resize_btn])

## Resize User Volume
Run the cell to render a dropdown list, select user volume to resize

In [7]:
import ipywidgets
from usage import *
from resize import *

user_volume = ipywidgets.Dropdown(
    options=get_user_volume_list().keys(),
    description='User vol:',
    disabled=False,
)
""
user_volume


Dropdown(description='User vol:', options=('claim-phuser-2d1', 'claim-phuser-2d2', 'claim-phuser-2d3', 'claim-…

Run the cell to render a input field of user volume size, specify a size for the user volume which we want to resize.

In [10]:
usage = get_user_volume_usages(user_volume.value)

num, unit = usage.get(user_volume.value).get('usage').get('data')

new_usage = ipywidgets.BoundedIntText(    
    value=num,
    min=num,
    max=9999,
    step=1,
    description='New Size:',
    disabled=False)

def resize_handler(self):
    resize_user_volume(user_volume.value, str(new_usage.value) + unit)

resize_btn = ipywidgets.Button(description="Update", button_style="danger")
resize_btn.on_click(resize_handler)

ipywidgets.HBox([
    ipywidgets.Label(value="Current Size: %s %s" % (num, unit)), new_usage, 
    ipywidgets.Label(value=unit), 
    resize_btn])